In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import rushd as rd
import scipy as sp
import seaborn as sns

from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'collaborators'/'birnbaum_steph'
output_path = rd.rootdir/'output'/'lenti_tcell'
cache_path = output_path/'data.gzip'

plates = pd.DataFrame({
    'data_path': [base_path/'2024-06-10 Galloway Exp 1'/'export', base_path/'2024-10-25 Galloway 2'/'export'],
    'yaml_path': [base_path/'2024-06-10 Galloway Exp 1'/'metadata.yaml', base_path/'2024-10-25 Galloway 2'/'export'/'metadata.yaml'],
})

for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)

In [ ]:
# Load data
data = pd.DataFrame()
channel_list = ['FITC-A', 'PE-A', 'APC-A750-A', 'PB450-A']

if cache_path.exists(): data = pd.read_parquet(cache_path)
else: 
    d1 = rd.flow.load_groups_with_metadata(plates.iloc[0].to_frame().transpose(), columns=channel_list)
    d2 = rd.flow.load_groups_with_metadata(plates.iloc[1].to_frame().transpose(), columns=channel_list)

    d1 = d1.rename({'FITC-A': 'mGL-A', 'PE-A': 'mRuby2-A', 'APC-A750-A': 'livedead-A'}, axis=1)
    d2 = d2.rename({'FITC-A': 'mGL-A', 'PE-A': 'mRuby2-A', 'PB450-A': 'livedead-A'}, axis=1)

    d1['biorep'] = 1
    d2['biorep'] = 2

    data = pd.concat([d1, d2], ignore_index=True)
    for c in ['mGL-A', 'mRuby2-A', 'livedead-A']: data = data[data[c]>0]
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

In [ ]:
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A',]
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['biorep'])[channel].apply(lambda x: x.quantile(0.9999))
gates.reset_index(inplace=True)

# Add metadata
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'
gates['exp'] = 'steph_' + gates['biorep'].astype(str)
data['exp'] = 'steph_' + data['biorep'].astype(str)

display(gates)

In [ ]:
# Gate data by marker expression
data = data.groupby(['exp','biorep'])[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)

# Gate live cells (livedead-A < gate)
data.loc[data['biorep']==1, 'live'] = data.loc[data['biorep']==1, 'livedead-A'] < 2e3
data.loc[data['biorep']==2, 'live'] = data.loc[data['biorep']==2, 'livedead-A'] < 7e4
display(data)

df = data[(data['expressing']) & (data['live']) & (data['construct']!='UT')]

In [ ]:
plot_df = df[df['moi']==1]
g = sns.displot(data=plot_df, x='mGL-A', y='mRuby2-A', row='biorep', col='construct', hue='dox',
                kind='kde', log_scale=True, common_norm=False, legend=False)

In [ ]:
plot_df = data[data['moi']==1]
g = sns.displot(data=plot_df, x='mGL-A', y='mRuby2-A', row='biorep', col='construct', hue='dox',
                kind='kde', log_scale=True, common_norm=False, legend=False)